# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 21 2022 11:40AM,253592,15,8931032-T,"Brookfield Pharmaceuticals, LLC",Released
1,Dec 21 2022 11:39AM,253591,17,8931023,"Brookfield Pharmaceuticals, LLC",Released
2,Dec 21 2022 11:27AM,253590,20,ATR-I000037,"HVL, LLC dba Atrium Innovations",Released
3,Dec 21 2022 11:21AM,253589,10,Eme-106755,Emerginnova,Released
4,Dec 21 2022 11:21AM,253589,10,Enova-11129,Emerginnova,Released
5,Dec 21 2022 11:21AM,253589,10,DNMD-29326,Emerginnova,Released
6,Dec 21 2022 11:21AM,253589,10,DNMD-29332,Emerginnova,Released
7,Dec 21 2022 10:57AM,253588,10,MSP216961,"Methapharm, Inc.",Released
8,Dec 21 2022 10:57AM,253588,10,MSP216962,"Methapharm, Inc.",Released
9,Dec 21 2022 10:57AM,253588,10,MSP216965,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,253588,Released,4
30,253589,Released,4
31,253590,Released,1
32,253591,Released,1
33,253592,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253588,NaN,NaN,4.0
253589,NaN,NaN,4.0
253590,NaN,NaN,1.0
253591,NaN,NaN,1.0
253592,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253494,0.0,18.0,24.0
253495,0.0,0.0,19.0
253497,0.0,13.0,2.0
253502,0.0,1.0,1.0
253505,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253494,0,18,24
253495,0,0,19
253497,0,13,2
253502,0,1,1
253505,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253494,0,18,24
1,253495,0,0,19
2,253497,0,13,2
3,253502,0,1,1
4,253505,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253494,,18,24
1,253495,,,19
2,253497,,13,2
3,253502,,1,1
4,253505,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 21 2022 11:40AM,253592,15,"Brookfield Pharmaceuticals, LLC"
1,Dec 21 2022 11:39AM,253591,17,"Brookfield Pharmaceuticals, LLC"
2,Dec 21 2022 11:27AM,253590,20,"HVL, LLC dba Atrium Innovations"
3,Dec 21 2022 11:21AM,253589,10,Emerginnova
7,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc."
11,Dec 21 2022 10:56AM,253587,10,"Citieffe, Inc."
12,Dec 21 2022 10:41AM,253585,10,"Citieffe, Inc."
13,Dec 21 2022 10:39AM,253586,20,"HVL, LLC dba Atrium Innovations"
15,Dec 21 2022 10:38AM,253584,19,ACG North America LLC
16,Dec 21 2022 10:35AM,253583,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 21 2022 11:40AM,253592,15,"Brookfield Pharmaceuticals, LLC",,,1
1,Dec 21 2022 11:39AM,253591,17,"Brookfield Pharmaceuticals, LLC",,,1
2,Dec 21 2022 11:27AM,253590,20,"HVL, LLC dba Atrium Innovations",,,1
3,Dec 21 2022 11:21AM,253589,10,Emerginnova,,,4
4,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",,,4
5,Dec 21 2022 10:56AM,253587,10,"Citieffe, Inc.",,,1
6,Dec 21 2022 10:41AM,253585,10,"Citieffe, Inc.",,,1
7,Dec 21 2022 10:39AM,253586,20,"HVL, LLC dba Atrium Innovations",,,2
8,Dec 21 2022 10:38AM,253584,19,ACG North America LLC,,,1
9,Dec 21 2022 10:35AM,253583,12,Hush Hush,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 11:40AM,253592,15,"Brookfield Pharmaceuticals, LLC",1,,
1,Dec 21 2022 11:39AM,253591,17,"Brookfield Pharmaceuticals, LLC",1,,
2,Dec 21 2022 11:27AM,253590,20,"HVL, LLC dba Atrium Innovations",1,,
3,Dec 21 2022 11:21AM,253589,10,Emerginnova,4,,
4,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",4,,
5,Dec 21 2022 10:56AM,253587,10,"Citieffe, Inc.",1,,
6,Dec 21 2022 10:41AM,253585,10,"Citieffe, Inc.",1,,
7,Dec 21 2022 10:39AM,253586,20,"HVL, LLC dba Atrium Innovations",2,,
8,Dec 21 2022 10:38AM,253584,19,ACG North America LLC,1,,
9,Dec 21 2022 10:35AM,253583,12,Hush Hush,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 11:40AM,253592,15,"Brookfield Pharmaceuticals, LLC",1,,
1,Dec 21 2022 11:39AM,253591,17,"Brookfield Pharmaceuticals, LLC",1,,
2,Dec 21 2022 11:27AM,253590,20,"HVL, LLC dba Atrium Innovations",1,,
3,Dec 21 2022 11:21AM,253589,10,Emerginnova,4,,
4,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 11:40AM,253592,15,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
1,Dec 21 2022 11:39AM,253591,17,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
2,Dec 21 2022 11:27AM,253590,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
3,Dec 21 2022 11:21AM,253589,10,Emerginnova,4.0,NaN,NaN
4,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 11:40AM,253592,15,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
1,Dec 21 2022 11:39AM,253591,17,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
2,Dec 21 2022 11:27AM,253590,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
3,Dec 21 2022 11:21AM,253589,10,Emerginnova,4.0,0.0,0.0
4,Dec 21 2022 10:57AM,253588,10,"Methapharm, Inc.",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3803283,56.0,63.0,0.0
12,253583,0.0,1.0,0.0
15,760581,44.0,18.0,0.0
16,507153,1.0,1.0,0.0
17,253591,1.0,0.0,0.0
19,1014282,89.0,2.0,1.0
20,507176,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3803283,56.0,63.0,0.0
1,12,253583,0.0,1.0,0.0
2,15,760581,44.0,18.0,0.0
3,16,507153,1.0,1.0,0.0
4,17,253591,1.0,0.0,0.0
5,19,1014282,89.0,2.0,1.0
6,20,507176,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,56.0,63.0,0.0
1,12,0.0,1.0,0.0
2,15,44.0,18.0,0.0
3,16,1.0,1.0,0.0
4,17,1.0,0.0,0.0
5,19,89.0,2.0,1.0
6,20,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,56.0
1,12,Released,0.0
2,15,Released,44.0
3,16,Released,1.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Executing,63.0,1.0,18.0,1.0,0.0,2.0,0.0
Released,56.0,0.0,44.0,1.0,1.0,89.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,63.0,1.0,18.0,1.0,0.0,2.0,0.0
2,Released,56.0,0.0,44.0,1.0,1.0,89.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,63.0,1.0,18.0,1.0,0.0,2.0,0.0
2,Released,56.0,0.0,44.0,1.0,1.0,89.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()